In [1]:
import geopandas as gpd
import pandas as pd
import folium
import pickle
import osmnx as ox


/home/janos/Documents/ITU/II_semester/venv/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_4904/2258308121.py:1: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/l

In [2]:
gdf = gpd.read_file("../data/data_all_gpd_v4.shp")
gdf.head()

,started_at,ended_at,duration,st_st_id,st_st_name,st_st_desc,st_st_lat,st_st_lon,en_st_id,en_st_name,en_st_desc,en_st_lat,en_st_lon,minutes,id,total_dist,waytypes_d,waytypes_s,geometry
0,2022-07-01 21:18:29.678000+00:00,2022-07-01 21:26:29.560000+00:00,479,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,7,5391,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
1,2022-07-02 08:39:02.618000+00:00,2022-07-02 08:47:54.958000+00:00,532,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,8,6044,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
2,2022-07-06 15:36:16.304000+00:00,2022-07-06 15:41:54.552000+00:00,338,408,Tøyen skole,forsiden av skolebygget,59.914943,10.773977,448,Oslo Plaza,ved rundkjøringen,59.912183,10.754434,5,31205,1502.2,0 3 7;3 13 2;13 20 3;20 50 2;50 82 3;82 91 7;,v:3.0 d:805.9 a:53.65;v:2.0 d:638.3 a:42.49;v:...,"LINESTRING (10.77395 59.91491, 10.77397 59.914..."
3,2022-07-02 08:39:04.505000+00:00,2022-07-02 08:48:36.381000+00:00,571,397,Storo Storsenter,langs Vitaminveien,59.946710,10.773805,421,Alexander Kiellands Plass,langs Maridalsveien,59.928067,10.751203,9,6045,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."
4,2022-07-02 12:38:43.229000+00:00,2022-07-02 12:49:29.189000+00:00,645,397,Storo Storsenter,langs Vitaminveien,59.946710,10.773805,421,Alexander Kiellands Plass,langs Maridalsveien,59.928067,10.751203,10,7493,2701.3,0 2 7;2 86 2;86 94 7;94 111 6;111 112 7;112 11...,v:2.0 d:1435.3 a:53.14;v:3.0 d:805.4 a:29.82;v...,"LINESTRING (10.77378 59.94668, 10.77369 59.946..."


In [3]:
len(gdf)

114925

In [4]:
st_coor = [[row['st_st_lat'], row['st_st_lon']] for _, row in gdf.iterrows()]
en_coor = [[row['en_st_lat'], row['en_st_lon']] for _, row in gdf.iterrows()]
agg_coor = st_coor
agg_coor.append(en_coor)

In [5]:
cood_df = pd.DataFrame()
st_lat = [row['st_st_lat'] for _, row in gdf.iterrows()]
en_lat = [row['en_st_lat'] for _, row in gdf.iterrows()]
st_lon = [row['st_st_lon'] for _, row in gdf.iterrows()]
en_lon = [row['en_st_lon'] for _, row in gdf.iterrows()]

lat = st_lat
lat.extend(en_lat)

lon = st_lon
lon.extend(en_lon)

cood_df['lon'] = lon
cood_df['lat'] = lat

In [6]:
cood_df = cood_df.drop_duplicates()
cood_df.head()

,lon,lat
0,10.773977,59.914943
3,10.773805,59.946710
33,10.757724,59.953411
34,10.753764,59.905124
35,10.703900,59.918578


In [7]:
cood_df.to_csv('../data/coordinates.csv', index=False)

In [8]:
with open('../data/bike_graph.pkl', 'rb') as f:
     G_bike = pickle.load(f)
     
gdf_edges = ox.graph_to_gdfs(G_bike, nodes=False, edges=True)

In [9]:
all_data = gpd.read_file("../data/data_all_gpd.shp")
all_data = all_data['geometry']
all_data = all_data.drop_duplicates()

In [10]:
m = folium.Map(location=[59.916823,10.747777], zoom_start=13)

for index, row in cood_df.iterrows():
    folium.Marker([row['lat'], row['lon']]).add_to(m)

folium.Choropleth(
    gdf,
    line_weight=1,
    line_color="blue"
).add_to(m)
    
folium.Choropleth(
    gdf_edges,
    line_weight=3,
    line_color="red"
).add_to(m)

m